In [1]:
import lxml
from urllib import request
from lxml import etree
import pandas as pd
import numpy as np
from selenium import webdriver # to allow js to load page before scrape

# Globals
baseurl = r'http://www.sharkattackdata.com'
headurl = baseurl + r'place/united_states_of_america'
 
shark_attack_data = []


ModuleNotFoundError: No module named 'selenium'

In [2]:
# Load the root page and scrape for data
browser = webdriver.Chrome(executable_path="C:/Users/red7740/Downloads/chromedriver.exe")
browser.get(headurl)
innerHTML = browser.execute_script("return document.body.innerHTML") #returns the inner HTML as a string
browser.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=62.0.3202.89)
  (Driver info: chromedriver=2.33.506120 (e3e53437346286c0bc2d2dc9aa4915ba81d9023f),platform=Windows NT 10.0.15063 x86_64)


In [ ]:
# Parse html string
myETree = lxml.etree.HTML(innerHTML)
# xpath for data
heads = myETree.xpath('//thead/tr/th')
attacks = myETree.xpath('//tbody/tr/td')
links = myETree.xpath('//tbody/tr/td/a/@href')

#convert column names to list
headings=[]
for h in heads:
    headings.append(h.text)

#convert data to list
data=[]
for a in attacks:
    data.append(a.text)

#convert links to list
links = list(links)


In [ ]:
headings.pop() # There is a blank column, so get rid of it and assign value 'url'
headings.append('url')

In [ ]:
# Massage Data into a pandas dataframe
#get some numbers...
ncols = len(headings)
nrows = int(len(data)/len(headings))

#Start with data -> numpy array -> reshape -> dataframe
data = np.array(data) #list-> numpy array
data = data.reshape(nrows,ncols)# reshape array
data = pd.DataFrame(data) #np.array -> dataframe

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1200x600')   

In [ ]:
def getHTML(url,browser):
    '''
    Returns inner html after JS executes
    '''
    # Navigate to url and scrape for data
    browser.get(url)
    innerHTML = browser.execute_script("return document.body.innerHTML") #returns the inner HTML as a string
    return innerHTML

In [50]:
#loop trough urls and grab data
detail_data = [] # data
detail_headers = [] # column names
count = 0 #counter for loop progress to display

browser = webdriver.Chrome(executable_path="C:/Users/red7740/Downloads/chromedriver.exe",chrome_options=options)#added headless

for url in links: # slice for debug. remove slice for full executable        
    # Parse html string
    myETree = lxml.etree.HTML(getHTML(baseurl+url,browser))
    # xpath for data
    dat = myETree.xpath('//tbody/tr/td')
    
    #convert data to list
    details=[]
    for d in dat:
        details.append(d.text)
        
    #append details of each record to data list
    detail_data.append(details)
    
    # First time through loop - grab headers
    if count == 0: #change count var to boolean if count-print removed
        # xpath for data
        dat = myETree.xpath('//tbody/tr/th')
        #convert data to list
        detail_headers=[]
        for d in dat:
            detail_headers.append(d.text)
            
    #print progress tracking for debugging - remove at final stage
    count += 1
    if (count % 50) == 0:
        print("Status: record " + str(count) + " completed. " + str(nrows - count) + " remaining")
        
# details -> dataframe     
detail_data = pd.DataFrame(detail_data)

browser.close()


Status: record 50 completed. 1607 remaining
Status: record 100 completed. 1557 remaining
Status: record 150 completed. 1507 remaining
Status: record 200 completed. 1457 remaining
Status: record 250 completed. 1407 remaining
Status: record 300 completed. 1357 remaining
Status: record 350 completed. 1307 remaining
Status: record 400 completed. 1257 remaining
Status: record 450 completed. 1207 remaining
Status: record 500 completed. 1157 remaining
Status: record 550 completed. 1107 remaining
Status: record 600 completed. 1057 remaining
Status: record 650 completed. 1007 remaining
Status: record 700 completed. 957 remaining
Status: record 750 completed. 907 remaining
Status: record 800 completed. 857 remaining
Status: record 850 completed. 807 remaining
Status: record 900 completed. 757 remaining
Status: record 950 completed. 707 remaining
Status: record 1000 completed. 657 remaining
Status: record 1050 completed. 607 remaining
Status: record 1100 completed. 557 remaining
Status: record 11

In [51]:
detail_data.columns = detail_headers


In [63]:
data3 = data.iloc[:,:6]
data3[:10]

,0,1,2,3,4,5
0,Unknown,Hawaii,Puna,None,Unprovoked,Non-fatal
1,Unknown,North Carolina,Somewhere between Hatteras and Beaufort,Swimming,Unprovoked,Non-fatal
2,Unknown,Florida,"Gadsden Point, Tampa Bay",Fishing,Unprovoked,Non-fatal
3,Unknown,Florida,"Palm Beach, Palm Beach County",Standing,Unprovoked,Non-fatal
4,Unknown,California,"Capistrano, Orange County",None,Unprovoked,Non-fatal
5,Unknown,Florida,"Mosquito Inlet (Ponce Inlet), Volusia County",Canoeing,Unprovoked,Fatal
6,Unknown,California,"LaJolla, San Diego County","Free diving, collecting sand dollars",Unprovoked,Non-fatal
7,Unknown,Florida,"Key West, Monroe County",Kitesurfing,Unprovoked,Non-fatal
8,Unknown,Florida,"Tampa Bay, Hillsborough County",Wade-fishing,Unprovoked,Non-fatal
9,Unknown,Florida,"Lost Tree Village, Palm Beach County",Surfing,Unprovoked,Non-fatal


In [52]:
detail_data[:15] # for debug -print the dataframe

,GSAF Case #,Date,Country,Area,Location,Activity,Name,Sex,Age,Injury,Time,Species,Investigator or Source,Fatal?,Incident Type
0,ND.0010,Unknown,None,None,Puna,None,A chiefess,Female,None,Ankle bitten,None,None,Captain W. Young,Not fatal,Unprovoked
1,ND.0014,Unknown,None,None,Somewhere between Hatteras and Beaufort,Swimming,"""youthful male""",Male,None,"""Lost leg""",None,None,"C. Creswell, GSAF; Wilmington Star, 7/17/1916",Not fatal,Unprovoked
2,ND.0019,Unknown,None,None,"Gadsden Point, Tampa Bay",Fishing,James Kelley,Male,None,2-inch lacerations,None,None,"T. Helm, p.219",Not fatal,Unprovoked
3,ND.0056,Unknown,None,None,"Palm Beach, Palm Beach County",Standing,Horton Chase,Male,None,Abrasions & bruises hip to ankle,None,None,"V.M. Coppleson (1956), p.255; R.F. Hutton",Not fatal,Unprovoked
4,ND.0060,Unknown,None,None,"Capistrano, Orange County",None,girl,Female,None,Leg injured,None,"White shark, 1,900-lb","B. Walton, Sun (San Bernardino), 6/19/1959",Not fatal,Unprovoked
5,ND.0078,Unknown,None,None,"Mosquito Inlet (Ponce Inlet), Volusia County",Canoeing,male,Male,None,FATAL,None,None,"W.H. Gregg, p.19; L. Schultz & M. Malin, ...",Fatal,Unprovoked
6,ND.0083,Unknown,None,None,"LaJolla, San Diego County","Free diving, collecting sand dollars",Charles Fleming,Male,None,Calf bitten,None,"Shovelnose guitarfish, adult male","C. Limbaugh in Sharks & Survival, pp.77-78",Not fatal,Unprovoked
7,ND.0097,Unknown,None,None,"Key West, Monroe County",Kitesurfing,Paul Menta,Male,None,Hand bitten,None,None,Internet,Not fatal,Unprovoked
8,ND-0109,Unknown,None,None,"Tampa Bay, Hillsborough County",Wade-fishing,Ed Snyder,Male,None,"No injury, shark rammed his back",None,None,Fishingworld.com,Not fatal,Unprovoked
9,ND-0122,Unknown,None,None,"Lost Tree Village, Palm Beach County",Surfing,C.M,Male,None,Legs bitten,None,None,M. Vorenberg,Not fatal,Unprovoked


In [53]:
data[:10]

,0,1,2,3,4,5,6
0,Unknown,Hawaii,Puna,None,Unprovoked,Non-fatal,None
1,Unknown,North Carolina,Somewhere between Hatteras and Beaufort,Swimming,Unprovoked,Non-fatal,None
2,Unknown,Florida,"Gadsden Point, Tampa Bay",Fishing,Unprovoked,Non-fatal,None
3,Unknown,Florida,"Palm Beach, Palm Beach County",Standing,Unprovoked,Non-fatal,None
4,Unknown,California,"Capistrano, Orange County",None,Unprovoked,Non-fatal,None
5,Unknown,Florida,"Mosquito Inlet (Ponce Inlet), Volusia County",Canoeing,Unprovoked,Fatal,None
6,Unknown,California,"LaJolla, San Diego County","Free diving, collecting sand dollars",Unprovoked,Non-fatal,None
7,Unknown,Florida,"Key West, Monroe County",Kitesurfing,Unprovoked,Non-fatal,None
8,Unknown,Florida,"Tampa Bay, Hillsborough County",Wade-fishing,Unprovoked,Non-fatal,None
9,Unknown,Florida,"Lost Tree Village, Palm Beach County",Surfing,Unprovoked,Non-fatal,None


In [57]:
detail_data2.to_csv('./detData2.csv')

In [56]:
detail_data2 = pd.DataFrame(detail_data)
detail_data.to_csv('./detData.csv')

data.to_csv('./data.csv')
data2 = pd.DataFrame(data)


In [67]:
#headings.pop()
data3.columns = headings
newdata = pd.concat([data3, detail_data2], axis=1) # links -> last column in data frame

In [68]:
newdata[:10]

,Date,Area,Location,Activity,Incident Type,Fatal?,GSAF Case #,Date,Country,Area,...,Activity,Name,Sex,Age,Injury,Time,Species,Investigator or Source,Fatal?,Incident Type
0,Unknown,Hawaii,Puna,None,Unprovoked,Non-fatal,ND.0010,Unknown,None,None,...,None,A chiefess,Female,None,Ankle bitten,None,None,Captain W. Young,Not fatal,Unprovoked
1,Unknown,North Carolina,Somewhere between Hatteras and Beaufort,Swimming,Unprovoked,Non-fatal,ND.0014,Unknown,None,None,...,Swimming,"""youthful male""",Male,None,"""Lost leg""",None,None,"C. Creswell, GSAF; Wilmington Star, 7/17/1916",Not fatal,Unprovoked
2,Unknown,Florida,"Gadsden Point, Tampa Bay",Fishing,Unprovoked,Non-fatal,ND.0019,Unknown,None,None,...,Fishing,James Kelley,Male,None,2-inch lacerations,None,None,"T. Helm, p.219",Not fatal,Unprovoked
3,Unknown,Florida,"Palm Beach, Palm Beach County",Standing,Unprovoked,Non-fatal,ND.0056,Unknown,None,None,...,Standing,Horton Chase,Male,None,Abrasions & bruises hip to ankle,None,None,"V.M. Coppleson (1956), p.255; R.F. Hutton",Not fatal,Unprovoked
4,Unknown,California,"Capistrano, Orange County",None,Unprovoked,Non-fatal,ND.0060,Unknown,None,None,...,None,girl,Female,None,Leg injured,None,"White shark, 1,900-lb","B. Walton, Sun (San Bernardino), 6/19/1959",Not fatal,Unprovoked
5,Unknown,Florida,"Mosquito Inlet (Ponce Inlet), Volusia County",Canoeing,Unprovoked,Fatal,ND.0078,Unknown,None,None,...,Canoeing,male,Male,None,FATAL,None,None,"W.H. Gregg, p.19; L. Schultz & M. Malin, ...",Fatal,Unprovoked
6,Unknown,California,"LaJolla, San Diego County","Free diving, collecting sand dollars",Unprovoked,Non-fatal,ND.0083,Unknown,None,None,...,"Free diving, collecting sand dollars",Charles Fleming,Male,None,Calf bitten,None,"Shovelnose guitarfish, adult male","C. Limbaugh in Sharks & Survival, pp.77-78",Not fatal,Unprovoked
7,Unknown,Florida,"Key West, Monroe County",Kitesurfing,Unprovoked,Non-fatal,ND.0097,Unknown,None,None,...,Kitesurfing,Paul Menta,Male,None,Hand bitten,None,None,Internet,Not fatal,Unprovoked
8,Unknown,Florida,"Tampa Bay, Hillsborough County",Wade-fishing,Unprovoked,Non-fatal,ND-0109,Unknown,None,None,...,Wade-fishing,Ed Snyder,Male,None,"No injury, shark rammed his back",None,None,Fishingworld.com,Not fatal,Unprovoked
9,Unknown,Florida,"Lost Tree Village, Palm Beach County",Surfing,Unprovoked,Non-fatal,ND-0122,Unknown,None,None,...,Surfing,C.M,Male,None,Legs bitten,None,None,M. Vorenberg,Not fatal,Unprovoked


In [70]:
newdata['Location'][:10]

,Location,Location
0,Puna,Puna
1,Somewhere between Hatteras and Beaufort,Somewhere between Hatteras and Beaufort
2,"Gadsden Point, Tampa Bay","Gadsden Point, Tampa Bay"
3,"Palm Beach, Palm Beach County","Palm Beach, Palm Beach County"
4,"Capistrano, Orange County","Capistrano, Orange County"
5,"Mosquito Inlet (Ponce Inlet), Volusia County","Mosquito Inlet (Ponce Inlet), Volusia County"
6,"LaJolla, San Diego County","LaJolla, San Diego County"
7,"Key West, Monroe County","Key West, Monroe County"
8,"Tampa Bay, Hillsborough County","Tampa Bay, Hillsborough County"
9,"Lost Tree Village, Palm Beach County","Lost Tree Village, Palm Beach County"


In [73]:

newdata.to_csv('./newattackdata.csv')